# Tutorial on manipulating SYN L2 AOD product

In [ ]:
from pathlib import Path
import xarray as xr
from xarray_eop.eop import open_eop_datatree

In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
SAMPLE_PATH = Path("./")
files = [ f for f in SAMPLE_PATH.glob("S3SYNAOD*.zarr")]
product = files[0]
product

### Open the whole product with datatree

In [ ]:
dt = open_eop_datatree(product)
dt

### Opening gridded measurement data with xarray

In [ ]:
group = "measurements"
# Open with xarray zarr engine
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
# Open with custom engine="eop"
ds=xr.open_dataset(SAMPLE_PATH / product, group=group, engine="eop")
aod550 = ds.aod_550
aod550

Underlying data is dask.array

In [ ]:
aod550.data

#### Simple raster plot

In [ ]:
aod550.plot()

#### Plot using the coordinates (lon,lat)
Note that in SYN AOD product, lat/lon are undefined when the data is missing, which is not correctly handled by matplotlib pcolormesh

In [ ]:
# Remove margins
aod550_dropna=aod550.dropna("columns",how="all")
aod550_dropna=aod550_dropna.dropna("rows",how="all")

# Fill remaining missing values
aod550_dropna["latitude"] = (["rows","columns"],aod550_dropna.latitude.bfill("columns").data)
aod550_dropna["longitude"] = (["rows","columns"],aod550_dropna.longitude.bfill("columns").data)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
ax = plt.axes()
aod550_dropna.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

#### Open uncertainties

In [ ]:
group = "quality"
ds=xr.open_dataset(SAMPLE_PATH / product / group, engine="zarr",chunks={})
ds